In [49]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import json
import datetime
import dill
import tqdm
from queue import PriorityQueue

In [50]:
class hier_softmax:
    def __init__(self, tree, contex_size, model):
        self._tree_tools = TreeTools()
        self.str2weight = {}
        #create a weight matrix and bias vector for each node in the tree
        for i, subtree in enumerate(self._tree_tools._get_subtrees(tree)):
            self.str2weight["softmax_node_"+str(i)+"_w"] = model.add_parameters((len(subtree), contex_size))
            self.str2weight["softmax_node_" + str(i) + "_b"] = model.add_parameters(len(subtree))
        
        #create a dictionary from each value to its path
        value_to_path_and_nodes_dict = {}
        for path, value in self._tree_tools._get_leaves_paths(tree):
            nodes = self._tree_tools._get_nodes(tree, path)
            value_to_path_and_nodes_dict[data.char2int[value]] = path, nodes
        self.value_to_path_and_nodes_dict = value_to_path_and_nodes_dict
        self.model = model
        self.tree = tree
    
    #get the loss on a given value (for training)
    def get_loss(self, context, value):
        loss = []
        path, nodes = self.value_to_path_and_nodes_dict[value]
        for p, n in zip(path, nodes):
            w = dy.parameter(self.str2weight["softmax_node_"+str(n)+"_w"])
            b = dy.parameter(self.str2weight["softmax_node_" + str(n) + "_b"])
            probs = tf.nn.softmax(w*context+b)
            #loss.append(-tf.math.log(dy.pick(probs, p)))
            print(probs)
            print(p)
        #return dy.esum(loss)

    #get the most likely
    def generate(self, context):
        best_value = None
        best_loss = float(100000)
        for value in self.value_to_path_and_nodes_dict:
            loss = self.get_loss(context, value)
            if loss < best_loss:
                best_loss = loss
                best_value = value
        return best_value

In [51]:
train_set = np.genfromtxt("data/{}_train_set.csv".format("SEG_Wavenet"), delimiter="\n", dtype=np.int64)
val_set = np.genfromtxt("data/{}_val_set.csv".format("SEG_Wavenet"), delimiter="\n", dtype=np.int64)

In [52]:
dataset = np.r_[train_set, val_set]
dataset

array([  0,   0,   0, ..., 897, 242, 961], dtype=int64)

## NOTE
Intermediate Nodes are indexed by **Preorder Traversal**

In [53]:
vocab_size = 16293
num_of_nodes = vocab_size - 1   # num of intermediate nodes

In [54]:
with open("outputs/tree_mapping.json", "r") as j:
    tree_mapping = json.load(j)     # category : [path, nodes in path]
tree_mapping = {int(i):j for i, j in tree_mapping.items()}      # As JSON converts key values to string
dict(list(tree_mapping.items())[:5]) 

{2192: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]],
 2396: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]],
 2647: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14]],
 1579: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14]],
 2247: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16]]}

In [55]:
tree_mapping[0], tree_mapping[1]

([[0, 1], [0, 1]], [[1, 0, 1], [0, 5436, 5437]])

In [56]:
path_matrix = np.full([vocab_size, num_of_nodes], -1, dtype=np.int32)
nodes_matrix = np.full([vocab_size, num_of_nodes], -1, dtype=np.int32)

In [57]:
for category, (path, nodes) in tree_mapping.items():
    path_matrix[category][:len(path)] = path
    nodes_matrix[category][:len(path)] = nodes

In [58]:
path_matrix.shape, nodes_matrix.shape

((16293, 16292), (16293, 16292))

In [100]:
class HierachicalSoftmax(keras.Model):
    def __init__(self, tree_mapping, vocab_size):
        super().__init__()

        self.vocab_size = vocab_size
        self.num_nodes = self.vocab_size - 1
        # decision_function = {1:1, 0:-1}
        self.decision_function = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(tf.constant([0, 1], dtype=tf.int32), tf.constant([-1, 1], dtype=tf.float32)), -1
        )      

        path_matrix = np.full([vocab_size, vocab_size-1], -1, dtype=np.int32)
        nodes_matrix = np.full([vocab_size, vocab_size-1], -1, dtype=np.int32)
        for category, (path, nodes) in tree_mapping.items():
            path_matrix[category][:len(path)] = path
            nodes_matrix[category][:len(path)] = nodes

        self.path_matrix = tf.constant(path_matrix)
        self.nodes_matrix = tf.constant(nodes_matrix)

    def build(self, input_shape):
        self.initializer = keras.initializers.GlorotNormal()
        self.softmax_weights = tf.Variable(self.initializer(shape=(self.num_nodes, input_shape[-1])))
    
    @tf.function
    def call(self, inputs):
        '''
        total_loss = tf.TensorArray(dtype=tf.float32, size=vocab_size)
        for i in tf.range(self.vocab_size):
            loss = self.get_loss(inputs, i)
            total_loss = total_loss.write(i, tf.reduce_prod(loss, axis=-1))
            tf.print(i)
        return total_loss.stack()
        '''
        y = tf.constant(0, dtype=tf.int32)
        return self.get_loss(inputs, y)

    @tf.function
    def get_loss(self, x, category):
        path = tf.gather(self.path_matrix, category)
        nodes = tf.gather(self.nodes_matrix, category)

        loss = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        for i in tf.range(self.num_nodes):
            n = tf.gather(nodes, i)
            if n == tf.constant(-1):
                break
            w = tf.gather(self.softmax_weights, n)
            d = self.decision_function.lookup(tf.gather(path, i))
            #sigma = tf.nn.softmax(w*x)
            sigma = tf.math.sigmoid(tf.tensordot(w*x, d, axes=0))
            tf.print(sigma) ###
            #loss = loss.write(i, tf.tensordot(sigma, d, axes=0))
            loss = loss.write(i, sigma)
        return tf.reduce_prod(loss.stack(), axis=-1)

    def train_step(self, data):
        x, y = data
        loss = self.get_loss(x, y)
        return loss

In [101]:
hsm = HierachicalSoftmax(tree_mapping, vocab_size)

In [102]:
x = tf.constant([0, 1], dtype=tf.float32)
x

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 1.], dtype=float32)>

In [103]:
y = tf.constant([0], dtype=tf.int32)
y

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>

In [104]:
hsm(x)

[0.5 0.497892112]
[0.5 0.49898687]


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.24894606, 0.24949344], dtype=float32)>

In [105]:
a = tf.constant([1, 2], shape=[2, 1])
b = tf.constant([3, 4])
a*b

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 4],
       [6, 8]])>